In [ ]:
import os
import shutil

# Define the paths to the mask and image folders
mask_folder = 'Dataset_mask'
image_folder = 'Dataset_edge'

# Create a new folder to store the combined images and masks
combined_folder = 'Dataset_combined'
os.makedirs(combined_folder, exist_ok=True)

# Copy all files from the mask folder to the combined folder
for mask_file in os.listdir(mask_folder):
    shutil.copy2(os.path.join(mask_folder, mask_file), combined_folder)

# Copy all files from the image folder to the combined folder
for image_file in os.listdir(image_folder):
    shutil.copy2(os.path.join(image_folder, image_file), combined_folder)

print("Masks and images have been combined and stored in the 'Combined' folder.")

In [ ]:
import os
import glob
from PIL import Image
import os
import glob
from PIL import Image

class ParkingLotDataset(Dataset):
    def __init__(self, root_img, root_msk, pairs=None, transforms=None):
        self.root_img = root_img
        self.root_msk = root_msk
        self.transforms = transforms

        if pairs is None:
            # Get all image files
            self.image_paths = sorted(glob.glob(os.path.join(root_img, '*.png')))

            # Get all mask files
            self.mask_paths = sorted(glob.glob(os.path.join(root_msk, '*.png')))

            # Pair image and mask files based on their filenames
            #self.pairs = [(image_path, mask_path) for image_path in self.image_paths for mask_path in self.mask_paths if os.path.splitext(os.path.basename(image_path))[0] == os.path.splitext(os.path.basename(mask_path))[0]]
            self.pairs = []

            for image_path in self.image_paths:
                image_filename = os.path.splitext(os.path.basename(image_path))[0]
                for mask_path in self.mask_paths:
                    mask_filename = os.path.splitext(os.path.basename(mask_path))[0]
                    if image_filename in mask_filename:
                        self.pairs.append((image_path, mask_path))
                        break  # Se trova una corrispondenza, passa al prossimo file immagine
        else:
            self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        image_path, mask_path = self.pairs[idx]

        # Load image
        image = Image.open(image_path)
        image_array = np.array(image)

        # Apply transformations
        if self.transforms:
            image_array = self.transforms(image_array)
            print(image_array.shape)
        

    


        return image_array, masks
        

        #merged_mask = np.zeros((2108, 2113), dtype=np.uint8)

        # Load and plot the masks
        #with open(mask_path, 'r') as f:
        #    masks_data = json.load(f)

        #for shape in masks_data['shapes']:
        #    points = np.array(shape['points'])
        #    mask = np.zeros((2108, 2113), dtype=np.uint8)
        #    cv2.fillPoly(mask, [points.astype(np.int32)], 1)
        #    _, binary_mask = cv2.threshold(mask, 0, 1, cv2.THRESH_BINARY)
        #    merged_mask += binary_mask

        #if self.transforms:
        #    masks = self.transforms(merged_mask)
        #    print(masks.shape)

        #print("Image shape:", image_array.shape)
        #print("Masks shape:", masks.shape)

plotting 1

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a DataLoader object called train_loader
num_samples = 10

# Iterate over batches from the DataLoader
for batch_idx, (images, masks) in enumerate(train_loader):
    # Get the number of samples in the current batch
    batch_size = images.size(0)

    # Iterate over the samples in the batch
    for i in range(batch_size):
        # Convert the image tensor to a NumPy array
        image_array = images[i].permute(1, 2, 0).numpy()

        # Convert the mask tensor to a NumPy array
        mask_array = masks[i].squeeze().numpy()

        # Plot the image and the combined mask
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(image_array, cmap='gray')
        plt.title(f'Image {batch_idx * batch_size + i + 1}')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(image_array, cmap='gray')
        plt.imshow(mask_array, cmap='gray', alpha=0.5)
        plt.title(f'Image {batch_idx * batch_size + i + 1} with Masks')
        plt.axis('off')

        plt.show()

        # Break after the first 10 samples
        if batch_idx * batch_size + i == num_samples - 1:
            break

    # Break after processing one batch
    if batch_idx == 0:
        break

plotting 2

In [ ]:
import matplotlib.pyplot as plt

# Get the first pair of image and masks from the dataset
image_path, mask_path = train_dataset.pairs[0]
print(image_path)
print(mask_path)
print(train_dataset.pairs[0])

# Load the image
image = Image.open(image_path)
image_array = np.array(image)

# Plot the image
plt.figure(figsize=(12, 6))
plt.imshow(image_array, cmap='gray')
plt.title('Image')

merged_mask = np.zeros((2108, 2113), dtype=np.uint8)

# Load and plot the masks
with open(mask_path, 'r') as f:
    masks_data = json.load(f)

for i, shape in enumerate(masks_data['shapes']):
    points = np.array(shape['points'])
    mask = np.zeros((2108, 2113), dtype=np.uint8)
    cv2.fillPoly(mask, [points.astype(np.int32)], 1)
    _, binary_mask = cv2.threshold(mask, 0, 1, cv2.THRESH_BINARY)
    merged_mask += binary_mask
plt.imshow(merged_mask, cmap='gray', alpha=0.0)
plt.title(f'Image with Masks')
plt.show()


CALCOLATORE DI DIMENSIONI IMMAGINI

In [ ]:
#controllo il massimo valore di altezza e larghezza per fare il padding
import cv2
import os

# Directory containing the images
data_dir = 'Dataset_edge_smaller'

# Initialize variables to store max height and width
max_height = 0
max_width = 0

# Iterate through all images in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)

        # Get height and width of the image
        height, width, _ = image.shape
        if(height > 500 or width > 500):
            print(filename)
            print(image.shape)

        # Update max_height and max_width if necessary
        if height > max_height:
            max_height = height
        if width > max_width:
            max_width = width

        

#Image with different max_height and max_width: Queretaro-TEC_parcheggi.png
print("Maximum Height:", max_height)
print("Maximum Width:", max_width)

Image resize

In [ ]:
import cv2
import os

# Directory containing the images
data_dir = 'resized_masks512'
# Directory to save resized images
resized_dir = 'resized_masks512_bis'

# Ensure the resized_dir exists
os.makedirs(resized_dir, exist_ok=True)

# Iterate through all images in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)

        # Get height and width of the image
        height, width, _ = image.shape

        # If the image is larger than 1000 in either dimension, resize it
        if height > 512 or width > 512:
            # Calculate the scale factor
            scale_factor = 0.5  # Reduce size by half

            # Calculate the new dimensions of the image
            new_width = int(width * scale_factor)
            new_height = int(height * scale_factor)

            # Resize the image
            image = cv2.resize(image, (new_width, new_height))

        # Save the image (resized or original) to the new directory
        new_image_path = os.path.join(resized_dir, filename)
        cv2.imwrite(new_image_path, image)
        

Mask resize and padding

In [ ]:
import cv2
import os

# Directory containing the images
data_dir = 'Resized_masks512'
# Directory to save resized images
resized_dir = 'Resizedmasks_smaller'

# Ensure the resized_dir exists
os.makedirs(resized_dir, exist_ok=True)

# Iterate through all images in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)

        # Get height and width of the image
        height, width, _ = image.shape

        # If the image is larger than 1000 in either dimension, resize it
        if height > 512 or width > 512:
            # Calculate the scale factor
            scale_factor = 0.5  # Reduce size by half

            # Calculate the new dimensions of the image
            new_width = int(width * scale_factor)
            new_height = int(height * scale_factor)

            # Resize the image
            image = cv2.resize(image, (new_width, new_height))

        ## MASK PADDING
        max_height = 512
        max_width = 512
        padding_height = max(0, max_height - image.shape[0])
        padding_width = max(0, max_width - image.shape[1])
        mask_padded = cv2.copyMakeBorder(image, 0, padding_height, 0, padding_width, cv2.BORDER_CONSTANT, value=0)
        # Save the image (resized or original) to the new directory
        new_image_path = os.path.join(resized_dir, filename)
        cv2.imwrite(new_image_path, mask_padded)


Mask resize and not pad

In [15]:
import os
from PIL import Image
import cv2
import numpy as np

# Directory contenente le immagini da ridimensionare
input_dir = "image_newmask"

# Directory di output dove verranno salvate le immagini ridimensionate
output_dir = "Resizedmasks_nopad"

# Dimensioni desiderate delle immagini ridimensionate
target_size = (512, 512)

# Assicurati che la directory di output esista, altrimenti creala
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Lista dei formati di immagine supportati da Pillow
supported_formats = [".jpg", ".jpeg", ".png", ".bmp", ".gif"]

# Loop attraverso tutti i file nella directory di input
for filename in os.listdir(input_dir):
    # Verifica se il file è un'immagine supportata
    if any(filename.lower().endswith(ext) for ext in supported_formats):
        # Percorso completo del file di input
        input_path = os.path.join(input_dir, filename)
        
        # Carica l'immagine utilizzando Pillow
        image = Image.open(input_path)
        
        # Ridimensiona l'immagine alla dimensione desiderata
        resized_image = image.resize(target_size)
        
        # Converti l'immagine ridimensionata in un array numpy
        resized_array = np.array(resized_image)

        output_image_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_image_path, resized_array)
            
print("Ridimensionamento completato.")


Ridimensionamento completato.


Dimension checker

In [33]:
for filename in os.listdir('RGB_dataset'):
    image_path = os.path.join('RGB_dataset', filename)
    image = cv2.imread(image_path)
    if image.shape[0] > 510 or image.shape[1] > 510:
        print(image_path)
        print(image.shape[0],image.shape[1])

RGB_dataset\101881731.png
512 512
RGB_dataset\103109948.png
512 512
RGB_dataset\103614205.png
512 512
RGB_dataset\104101927.png
512 512
RGB_dataset\104357814.png
512 512
RGB_dataset\104357815.png
512 512
RGB_dataset\104357817.png
512 512
RGB_dataset\104907575.png
512 512
RGB_dataset\116016718.png
512 512
RGB_dataset\119422952.png
512 512
RGB_dataset\119917133.png
512 512
RGB_dataset\126381586.png
512 512
RGB_dataset\126381588.png
512 512
RGB_dataset\126381594.png
512 512
RGB_dataset\127649594.png
512 512
RGB_dataset\127649606.png
512 512
RGB_dataset\130730813.png
512 512
RGB_dataset\132613254.png
512 512
RGB_dataset\134714700.png
512 512
RGB_dataset\136128142.png
512 512
RGB_dataset\136128143.png
512 512
RGB_dataset\136128144.png
512 512
RGB_dataset\137007454.png
512 512
RGB_dataset\137007455.png
512 512
RGB_dataset\139146552.png
512 512
RGB_dataset\139146568.png
512 512
RGB_dataset\144384815.png
512 512
RGB_dataset\144848258.png
512 512
RGB_dataset\147397541.png
512 512
RGB_dataset\14

512x512 no padding

In [13]:
import os
from PIL import Image
import cv2
import numpy as np

# Directory contenente le immagini da ridimensionare
input_dir = "dataset"

# Directory di output dove verranno salvate le immagini ridimensionate
output_dir = "Resize_no_padding"

# Dimensioni desiderate delle immagini ridimensionate
target_size = (512, 512)

# Assicurati che la directory di output esista, altrimenti creala
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Lista dei formati di immagine supportati da Pillow
supported_formats = [".jpg", ".jpeg", ".png", ".bmp", ".gif"]

# Loop attraverso tutti i file nella directory di input
for filename in os.listdir(input_dir):
    # Verifica se il file è un'immagine supportata
    if any(filename.lower().endswith(ext) for ext in supported_formats):
        # Percorso completo del file di input
        input_path = os.path.join(input_dir, filename)
        
        # Carica l'immagine utilizzando Pillow
        image = Image.open(input_path)
        
        # Ridimensiona l'immagine alla dimensione desiderata
        resized_image = image.resize(target_size)
        
        # Converti l'immagine ridimensionata in un array numpy
        resized_array = np.array(resized_image)
        
        if "_testing" in filename or "_training" in filename:
            # Rinominare il file rimuovendo "_testing" o "_training" dal nome
            new_filename = filename.replace("_testing", "").replace("_training", "")
            
            print(f"Rinominato il file {filename} in {new_filename}")
            # Salva l'immagine del bordo nella cartella di output
            output_image_path = os.path.join(output_dir, new_filename)
            cv2.imwrite(output_image_path, resized_array)
            
print("Ridimensionamento completato.")


Rinominato il file 101881731_training.png in 101881731.png
Rinominato il file 103109948_testing.png in 103109948.png
Rinominato il file 103614205_training.png in 103614205.png
Rinominato il file 104101927_testing.png in 104101927.png
Rinominato il file 104357814_training.png in 104357814.png
Rinominato il file 104357815_training.png in 104357815.png
Rinominato il file 104357817_training.png in 104357817.png
Rinominato il file 104907575_testing.png in 104907575.png
Rinominato il file 116016718_training.png in 116016718.png
Rinominato il file 119422952_training.png in 119422952.png
Rinominato il file 119917133_training.png in 119917133.png
Rinominato il file 126381586_training.png in 126381586.png
Rinominato il file 126381588_training.png in 126381588.png
Rinominato il file 126381594_training.png in 126381594.png
Rinominato il file 127649594_training.png in 127649594.png
Rinominato il file 127649606_testing.png in 127649606.png
Rinominato il file 130730813_training.png in 130730813.png
R

Store canny edge

In [ ]:
dataset_path = 'ResizedDataset_even_smaller'
edge_path = 'Dataset_edge_smaller'

os.makedirs(edge_path, exist_ok=True)

def get_edge_image(image, low_threshold, high_threshold):
    # Convert the image to grayscale if it's not already
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Canny edge detection
    edges = cv2.Canny(gray, low_threshold, high_threshold)
    
    return edges
#edge detection + padding immagini
for filename in os.listdir(dataset_path):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        
        input_image_path = os.path.join(dataset_path, filename)
        image = cv2.imread(input_image_path)
        # Ottieni l'immagine del bordo
        edge_image = get_edge_image(image, low_threshold=200, high_threshold=300)
        
        # Apply padding to the edge image
        max_height = 512
        max_width = 512
        padding_height = max(0, max_height - edge_image.shape[0])
        padding_width = max(0, max_width - edge_image.shape[1])
        edge_image_padded = cv2.copyMakeBorder(edge_image, 0, padding_height, 0, padding_width, cv2.BORDER_CONSTANT, value=0)
        #Verificare se il nome del file contiene "_testing" o "_training"
        if "_testing" in filename or "_training" in filename:
            # Rinominare il file rimuovendo "_testing" o "_training" dal nome
            new_filename = filename.replace("_testing", "").replace("_training", "")
            
            print(f"Rinominato il file {filename} in {new_filename}")
            # Salva l'immagine del bordo nella cartella di output
            output_image_path = os.path.join(edge_path, new_filename)
            cv2.imwrite(output_image_path, edge_image_padded)

print("Il rilevamento dei bordi è stato applicato a tutte le immagini nella cartella.")


RGB

In [32]:
dataset_path = 'ResizedDataset_even_smaller'
edge_path = 'RGB_dataset'

os.makedirs(edge_path, exist_ok=True)

for filename in os.listdir(dataset_path):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        
        input_image_path = os.path.join(dataset_path, filename)
        image = cv2.imread(input_image_path)

        # Apply padding to the edge image
        max_height = 512
        max_width = 512
        padding_height = max(0, max_height - image.shape[0])
        padding_width = max(0, max_width - image.shape[1])
        image_padded = cv2.copyMakeBorder(image, 0, padding_height, 0, padding_width, cv2.BORDER_CONSTANT, value=0)
        #Verificare se il nome del file contiene "_testing" o "_training"
        if "_testing" in filename or "_training" in filename:
            # Rinominare il file rimuovendo "_testing" o "_training" dal nome
            new_filename = filename.replace("_testing", "").replace("_training", "")
            
            print(f"Rinominato il file {filename} in {new_filename}")
            # Salva l'immagine del bordo nella cartella di output
            output_image_path = os.path.join(edge_path, new_filename)
            cv2.imwrite(output_image_path, image_padded)

Rinominato il file 101881731_training.png in 101881731.png
Rinominato il file 103109948_testing.png in 103109948.png
Rinominato il file 103614205_training.png in 103614205.png
Rinominato il file 104101927_testing.png in 104101927.png
Rinominato il file 104357814_training.png in 104357814.png
Rinominato il file 104357815_training.png in 104357815.png
Rinominato il file 104357817_training.png in 104357817.png
Rinominato il file 104907575_testing.png in 104907575.png
Rinominato il file 116016718_training.png in 116016718.png
Rinominato il file 119422952_training.png in 119422952.png
Rinominato il file 119917133_training.png in 119917133.png
Rinominato il file 126381586_training.png in 126381586.png
Rinominato il file 126381588_training.png in 126381588.png
Rinominato il file 126381594_training.png in 126381594.png
Rinominato il file 127649594_training.png in 127649594.png
Rinominato il file 127649606_testing.png in 127649606.png
Rinominato il file 130730813_training.png in 130730813.png
R

RGB normalizzazione

In [38]:
from PIL import Image
import os
import numpy as np

# Definisci il percorso della cartella contenente le immagini
folder_path = 'RGB_dataset'

# Inizializza le liste per i canali RGB
red_values = []
green_values = []
blue_values = []

# Itera attraverso le immagini nella cartella
for filename in os.listdir(folder_path):
    # Verifica se il file è un'immagine
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Carica l'immagine
        image = Image.open(os.path.join(folder_path, filename))
        # Estrai i valori dei pixel
        pixels = np.array(image)
        # Calcola la media dei valori dei pixel per ciascun canale di colore
        red_values.append(np.mean(pixels[:,:,0]))
        green_values.append(np.mean(pixels[:,:,1]))
        blue_values.append(np.mean(pixels[:,:,2]))

# Calcola la media e la deviazione standard per ogni canale di colore
mean_red = np.mean(red_values)
mean_green = np.mean(green_values)
mean_blue = np.mean(blue_values)

std_red = np.std(red_values)
std_green = np.std(green_values)
std_blue = np.std(blue_values)

print("Mean Red:", mean_red)
print("Mean Green:", mean_green)
print("Mean Blue:", mean_blue)

print("Std Red:", std_red)
print("Std Green:", std_green)
print("Std Blue:", std_blue)


Mean Red: 35.54499132537842
Mean Green: 35.16268335723877
Mean Blue: 33.40791042327881
Std Red: 21.848256784304716
Std Green: 21.59419671163517
Std Blue: 20.93842322158817


RGB normalizzazione 2

In [3]:
import numpy as np
from PIL import Image
import os
import glob

def calculate_mean_std(image_paths):
    pixel_values = []
    for image_path in image_paths:
        image = Image.open(image_path).convert('RGB')
        image_array = np.array(image)
        pixel_values.extend(image_array.reshape(-1, 3))

    pixel_values = np.array(pixel_values, dtype=np.float32)
    mean = pixel_values.mean(axis=0) / 255.0
    std = pixel_values.std(axis=0) / 255.0
    return mean, std

# Get all image paths
image_paths = sorted(glob.glob(os.path.join('Dataset', '*.png')))

# Calculate mean and std
mean, std = calculate_mean_std(image_paths)
print(f"Mean: {mean}")
print(f"Std: {std}")

Mean: [0.12028961 0.12028961 0.12028961]
Std: [0.27084008 0.2638215  0.25238606]


In [5]:
import os
from PIL import Image
import numpy as np

def calculate_mean_std(image_dir):
    image_paths = []
    for filename in os.listdir(image_dir):
        image_path = os.path.join(image_dir, filename)
        if os.path.isfile(image_path):
            image_paths.append(image_path)

    pixel_values = []
    for image_path in image_paths:
        image = Image.open(image_path).convert('RGB')
        image_array = np.array(image, dtype=np.uint8)  # Ensure pixel values are integers
        pixel_values.extend(image_array.reshape(-1, 3))
    pixel_values = np.array(pixel_values, dtype=np.uint8)  # Ensure pixel values are integers
    mean = pixel_values.mean(axis=0)
    std = pixel_values.std(axis=0)
    return mean, std

# Assuming your images are in a directory named 'Dataset' in the current working directory
image_dir = 'RGB_dataset'
mean, std = calculate_mean_std(image_dir)
print(f"Mean: {mean}")
print(f"Std: {std}")

Mean: [35.54499133 35.16268336 33.40791042]
Std: [57.87239911 56.89318723 54.54667448]
